In [ ]:
import numpy as np

## 付録F　電気ヒーター給湯機及び電気ヒーター給湯温水暖房機の給湯部 / Appendix F Water heater of electric heater water heater and electric heater water heater hot water heater

### F.1 記号及び単位 / Symbol and unit

省略

Omission

### F.2 消費電力量 / Power consumption

1日当たりの沸き上げ時間$ \Delta t_{wh} $は8時間とする。

Boiling time $ \Delta t_ {wh} $ per day is 8 hours.

$$\large
E_{E,hs,d,t} = \left\{ \begin{array}{ll}
    (a \times \theta_{ex,Ave,d} + b \times L''_{d} + c) \div 3.6 \div \Delta t_{wh} & \big(t=0～6 \big)\\ 
    0 & \big(t=7～22 \big)\\ 
    (a \times \theta_{ex,Ave,d+1} + b \times L''_{d+1} + c) \div 3.6 \div \Delta t_{wh} & \big(t=23 \big)\\ 
    \end{array}
\right.
$$

$E_{E,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの給湯機の消費電力量(kWh/h) / Water heater power consumption per hour at time $ t $ on date $ d $  
$L''_{d}$：日付$d$における1 日当たりの太陽熱補正給湯熱負荷(MJ/d) / One day solar heat correction hot water supply heat load on date $ d $  
$\Delta t_{wh}$：1日当たりの沸き上げ時間(h/d) / Boiling time per day  
$\theta_{ex,Ave,d}$：日付$d$における日平均外気温度(℃) / Daily average outside temperature on date $ d $  
$a, b, c$：係数(-) / coefficient  

日付$d$が12月31日の場合、日付$d+1$は1月1日と読み替える。

If the date $ d $ is December 31, the date $ d + 1 $ is read as January 1st.

<div style="text-align: center;font-weight: bold;">表　係数<br>Table: Coefficients</div>

<table>
<tr><th>a</th><td>-0.13801</td></tr>
<tr><th>b</th><td>0.975853</td></tr>
<tr><th>c</th><td>13.7426</td></tr>
</table>


In [ ]:
# 1時間当たりの給湯機の消費電力量 (1)
def get_E_E_hs_d_t(L_dashdash_k_d_t, L_dashdash_s_d_t, L_dashdash_w_d_t, L_dashdash_b1_d_t, L_dashdash_b2_d_t,
                   L_dashdash_ba1_d_t, L_dashdash_ba2_d_t,
                   Theta_ex_Ave_d):
    """
    :param L_dashdash_k_d_t: 1時間当たりの台所水栓における太陽熱補正給湯熱負荷 (MJ/h)
    :param L_dashdash_s_d_t: 1時間当たりの浴室シャワー水栓における太陽熱補正給湯負荷 (MJ/h)
    :param L_dashdash_w_d_t: 1時間当たりの洗面水栓における太陽熱補正給湯負荷 (MJ/h)
    :param L_dashdash_b1_d_t: 1時間当たりの浴槽水栓湯はり時における太陽熱補正給湯負荷 (MJ/h)
    :param L_dashdash_b2_d_t: 1時間当たりの浴槽自動湯はり時における太陽熱補正給湯負荷 (MJ/h)
    :param L_dashdash_ba1_d_t: 1時間当たりの浴槽水栓さし湯時における太陽熱補正給湯負荷 (MJ/h)
    :param L_dashdash_ba2_d_t: 1時間当たりの浴槽追焚時における太陽熱補正給湯負荷 (MJ/h)
    :param Theta_ex_Ave_d: 日平均外気温度 (℃)
    :return: 1時間当たりの給湯機の消費電力量 (kWh/h)
    """
    # 表F.2 係数
    a, b, c = (
        -0.13801,
        0.975853,
        13.7426
    )

    # 1日当たりの太陽熱補正給湯熱負荷
    L_dashdash_k_d = get_L_dashdash_k_d(L_dashdash_k_d_t)
    L_dashdash_s_d = get_L_dashdash_s_d(L_dashdash_s_d_t)
    L_dashdash_w_d = get_L_dashdash_w_d(L_dashdash_w_d_t)
    L_dashdash_b1_d = get_L_dashdash_b1_d(L_dashdash_b1_d_t)
    L_dashdash_b2_d = get_L_dashdash_b2_d(L_dashdash_b2_d_t)
    L_dashdash_ba1_d = get_L_dashdash_ba1_d(L_dashdash_ba1_d_t)
    L_dashdash_ba2_d = get_L_dashdash_ba2_d(L_dashdash_ba2_d_t)
    L_dashdash_d = get_L_dashdash_d(L_dashdash_k_d, L_dashdash_s_d, L_dashdash_w_d, L_dashdash_b1_d,
                                    L_dashdash_b2_d, L_dashdash_ba1_d,
                                    L_dashdash_ba2_d)

    Theta_ex_Ave_d_t = np.repeat(Theta_ex_Ave_d, 24)
    Theta_ex_Ave_d_t_p1 = np.roll(Theta_ex_Ave_d_t, -24)
    L_dashdash_d = np.repeat(L_dashdash_d, 24)

    E_E_hs_d_t = np.zeros(24 * 365)
    t_wh = 8

    # (1) t == 0 - 6
    f1 = np.tile(np.arange(24) <= 6, 365)
    E_E_hs_d_t[f1] = (a * Theta_ex_Ave_d_t[f1] + b * L_dashdash_d[f1] + c) / 3.6 / t_wh

    # (2) t == 7-22
    f2 = np.tile(np.logical_and(7 <= np.arange(24), np.arange(24) <= 22), 365)
    E_E_hs_d_t[f2] = 0

    # (3) t == 23
    f3 = np.tile(np.arange(24) == 23, 365)
    E_E_hs_d_t[f3] = (a * Theta_ex_Ave_d_t_p1[f3] + b * L_dashdash_d[f3] + c) / 3.6 / t_wh

    return E_E_hs_d_t

### F.3 ガス消費量 / Gas consumption

日付$d$の時刻$t$における 1時間当たりの給湯機のガス消費量$E_{G,hs,d,t}$は0とする。

The gas consumption $ E_ {G, hs, d, t} $ of the water heater per hour at the time $ t $ of the date $ d $ is 0.

In [ ]:
# 1日当たりの給湯機のガス消費量
def get_E_G_hs():
    # 1日当たりの給湯機のガス消費量は0とする
    return np.zeros(24 * 365)

### F.4 灯油消費量 / Kerosene consumption

日付$d$の時刻$t$における 1時間当たりの給湯機の灯油消費量$E_{K,hs,d,t}$は0とする。

The kerosene consumption $ E_ {K, hs, d, t} $ of the water heater per hour at the time $ t $ of the date $ d $ is 0.

In [ ]:
# 1日当たりの給湯機の灯油消費量
def get_E_K_hs():
    # 1日当たりの給湯機の灯油消費量は0とする
    return np.zeros(24 * 365)

### F.5 １日当たりの太陽熱補正給湯熱負荷 / Solar heat correction hot water supply heat load per day

$$\large
\displaystyle
L''_{d} = L''_{k,d}+L''_{s,d}+L''_{w,d}+L''_{b1,d}+L''_{b2,d}+L''_{ba1,d}+L''_{ba2,d}
$$

$ L''_{d} $：日付$d$における1 日当たりの太陽熱補正給湯熱負荷(MJ/d) / One day of solar correction hot water supply heat load at date $ d $  
$ L''_{k,d} $：日付$d$における1 日当たりの台所水栓における太陽熱補正給湯熱負荷(MJ/d) / Solar correction water supply heat load on a kitchen faucet per day at a date $ d $  
$ L''_{s,d} $：日付$d$における1 日当たりの浴室シャワー水栓における太陽熱補正給湯熱負荷(MJ/d) / Solar correction water supply heat load at one day bathroom shower faucet at date $ d $  
$ L''_{w,d} $：日付$d$における1 日当たりの洗面水栓における太陽熱補正給湯熱負荷(MJ/d) / solar correction hot water supply heat load in a single day flush faucet at date $ d $  
$ L''_{b1,d} $：日付$d$における1 日当たりの浴槽水栓湯はり時における太陽熱補正給湯熱負荷(MJ/d) / Solar correction water supply heat load during one day bathtub faucet hot water at $ d $ date  
$ L''_{b2,d} $：日付$d$における1 日当たりの浴槽自動湯はり時における太陽熱補正給湯熱負荷(MJ/d) / Solar correction hot water supply heat load during one day bathtub automatic water heating at date $ d $  
$ L''_{ba1,d} $：日付$d$における1 日当たりの浴槽水栓さし湯時における太陽熱補正給湯熱負荷(MJ/d) / Solar correction water supply heat load during one day bathtub faucet breaking time at date $ d $  
$ L''_{ba2,d} $：日付$d$における1 日当たりの浴槽追焚時における太陽熱補正給湯熱負荷(MJ/d) / Solar heat correction hot water supply heat load at the time of bathtub remembrance on one day in date $ d $  

In [ ]:
# 1日当たりの太陽熱補正給湯熱負荷 (2)
def get_L_dashdash_d(L_dashdash_k_d, L_dashdash_s_d, L_dashdash_w_d, L_dashdash_b1_d, L_dashdash_b2_d, L_dashdash_ba1_d,
                     L_dashdash_ba2_d):
    """
    :param L_dashdash_k_d: 1日当たりの台所水栓における太陽熱補正給湯熱負荷 (MJ/d)
    :param L_dashdash_s_d: 1日当たりの浴室シャワー水栓における太陽熱補正給湯負荷 (MJ/d)
    :param L_dashdash_w_d: 1日当たりの洗面水栓における太陽熱補正給湯負荷 (MJ/d)
    :param L_dashdash_b1_d: 1日当たりの浴槽水栓湯はり時における太陽熱補正給湯負荷 (MJ/d)
    :param L_dashdash_b2_d: 1日当たりの浴槽自動湯はり時における太陽熱補正給湯負荷 (MJ/d)
    :param L_dashdash_ba1_d: 1日当たりの浴槽水栓さし湯時における太陽熱補正給湯負荷 (MJ/d)
    :param L_dashdash_ba2_d: 1日当たりの浴槽追焚時における太陽熱補正給湯負荷 (MJ/d)
    :return: 1日当たりの太陽熱補正給湯熱負荷 (MJ/d)
    """
    return L_dashdash_k_d \
           + L_dashdash_s_d \
           + L_dashdash_w_d \
           + L_dashdash_b1_d \
           + L_dashdash_b2_d \
           + L_dashdash_ba1_d \
           + L_dashdash_ba2_d

$$\large
L''_{k,d} = \sum_{t=0}^{23} L''_{k,d,t}
$$

$$\large
L''_{s,d} = \sum_{t=0}^{23} L''_{s,d,t}
$$

$$\large
L''_{w,d} = \sum_{t=0}^{23} L''_{w,d,t}
$$

$$\large
L''_{b1,d} = \sum_{t=0}^{23} L''_{b1,d,t}
$$

$$\large
L''_{b2,d} = \sum_{t=0}^{23} L''_{b2,d,t}
$$

$$\large
L''_{ba1,d} = \sum_{t=0}^{23} L''_{ba1,d,t}
$$

$$\large
L''_{ba2,d} = \sum_{t=0}^{23} L''_{ba2,d,t}
$$

$L''_{k,d}$：：日付$d$における1 日当たりの台所水栓における太陽熱補正給湯熱負荷(MJ/d) / Solar heat correction hot water supply heat load on a kitchen faucet per day at date $ d $  
$L''_{s,d}$：：日付$d$における1 日当たりの浴室シャワー水栓における太陽熱補正給湯熱負荷(MJ/d) / Solar heat correction hot water supply heat load at one day bathroom shower faucet at date $ d $  
$L''_{w,d}$：：日付$d$における1 日当たりの洗面水栓における太陽熱補正給湯熱負荷(MJ/d) / Solar heat correction hot water supply heat load in a single day water faucet at date $ d $  
$L''_{b1,d}$：：日付$d$における1 日当たりの浴槽水栓湯はり時における太陽熱補正給湯熱負荷(MJ/d) / Solar heat correction hot water supply heat load at the time of one day bathtub faucet hot water at date $ d $  
$L''_{b2,d}$：：日付$d$における1 日当たりの浴槽自動湯はり時における太陽熱補正給湯熱負荷(MJ/d) / Solar heat correction hot water supply heat load at the time of one day bathtub automatic bath heating on date $ d $  
$L''_{ba1,d}$：日付$d$における1 日当たりの浴槽水栓さし湯時における太陽熱補正給湯熱負荷(MJ/d) / Solar heat correction hot water supply heat load at the time of one day bathtub bathtub faucet at date $ d $  
$L''_{ba2,d}$：日付$d$における1 日当たりの浴槽追焚時における太陽熱補正給湯熱負荷(MJ/d) / Solar heat correction hot water supply heat load at the time of one day bathtub remembrance at date $ d $  
$ L''_{k,d,t} $：日付$d$の時刻$t$における 1時間当たりの台所水栓における太陽熱補正給湯熱負荷(MJ/h) / Solar heat correction hot water supply heat load on kitchen faucet per hour at time $ t $ on date $ d $  
$ L''_{s,d,t} $：日付$d$の時刻$t$における 1時間当たりの浴室シャワー水栓における太陽熱補正給湯熱負荷(MJ/h) / Solar heat correction hot water supply heat load in bathroom shower faucet per hour at time $ t $ of date $ d $  
$ L''_{w,d,t} $：日付$d$の時刻$t$における 1時間当たりの洗面水栓における太陽熱補正給湯熱負荷(MJ/h) / Solar heat correction hot water supply heat load in the faucet per hour at time $ t $ of date $ d $    
$ L''_{b1,d,t} $：日付$d$の時刻$t$における 1時間当たりの浴槽水栓湯はり時における太陽熱補正給湯熱負荷(MJ/h) / Solar heat correction hot water supply heat load at the time of bathtub faucet bathing per hour at time $ t $ of date $ d $  
$ L''_{b2,d,t} $：日付$d$の時刻$t$における 1時間当たりの浴槽自動湯はり時における太陽熱補正給湯熱負荷(MJ/h) / Solar heat correction hot water supply heat load at the time of bathtub automatic water heating per hour at time $ t $ of date $ d $  
$ L''_{ba1,d,t} $：日付$d$の時刻$t$における 1時間当たりの浴槽水栓さし湯時における太陽熱補正給湯熱負荷(MJ/h) / Solar heat correction hot water supply heat load at the time of bathtub faucet breaking water per hour at time $ t $ of date $ d $  
$ L''_{ba2,d,t} $：日付$d$の時刻$t$における 1時間当たりの浴槽追焚時における太陽熱補正給湯熱負荷(MJ/h) / Solar heat correction hot water supply heat load at the time of tub recollection per hour at time $ t $ of date $ d $
  

In [ ]:
# 1日当たりの台所水栓における太陽熱補正給湯熱負荷 (MJ/d)
def get_L_dashdash_k_d(L_dashdash_k_d_t):
    """
    :param L_dashdash_k_d_t: 1時間当たりの台所水栓における太陽熱補正給湯熱負荷 (MJ/h)
    :return: 1日当たりの台所水栓における太陽熱補正給湯熱負荷 (MJ/d)
    """
    return np.sum(L_dashdash_k_d_t.reshape((365, 24)), axis=1)


# 1日当たりの浴室シャワー水栓における太陽熱補正給湯負荷 (MJ/d)
def get_L_dashdash_s_d(L_dashdash_s_d_t):
    """
    :param L_dashdash_s_d_t: 1時間当たりの浴室シャワー水栓における太陽熱補正給湯負荷 (MJ/h)
    :return: 1日当たりの浴室シャワー水栓における太陽熱補正給湯負荷 (MJ/d)
    """
    return np.sum(L_dashdash_s_d_t.reshape((365, 24)), axis=1)


# 1日当たりの洗面水栓における太陽熱補正給湯負荷 (MJ/d)
def get_L_dashdash_w_d(L_dashdash_w_d_t):
    """
    :param L_dashdash_w_d_t: 1時間当たりの洗面水栓における太陽熱補正給湯負荷 (MJ/h)
    :return: 1日当たりの洗面水栓における太陽熱補正給湯負荷 (MJ/d)
    """
    return np.sum(L_dashdash_w_d_t.reshape((365, 24)), axis=1)


# 1日当たりの浴槽水栓湯はり時における太陽熱補正給湯負荷 (MJ/d)
def get_L_dashdash_b1_d(L_dashdash_b1_d_t):
    """
    :param L_dashdash_b1_d_t: 1時間当たりの浴槽水栓湯はり時における太陽熱補正給湯負荷 (MJ/h)
    :return: 1日当たりの浴槽水栓湯はり時における太陽熱補正給湯負荷 (MJ/d)
    """
    return np.sum(L_dashdash_b1_d_t.reshape((365, 24)), axis=1)


# 1日当たりの浴槽自動湯はり時における太陽熱補正給湯負荷 (MJ/d)
def get_L_dashdash_b2_d(L_dashdash_b2_d_t):
    """
    :param L_dashdash_b2_d_t: 1時間当たりの浴槽自動湯はり時における太陽熱補正給湯負荷 (MJ/h)
    :return: 1日当たりの浴槽自動湯はり時における太陽熱補正給湯負荷 (MJ/d)
    """
    return np.sum(L_dashdash_b2_d_t.reshape((365, 24)), axis=1)


# 1日当たりの浴槽水栓さし湯時における太陽熱補正給湯負荷 (MJ/d)
def get_L_dashdash_ba1_d(L_dashdash_ba1_d_t):
    """
    :param L_dashdash_ba1_d_t: 1時間当たりの浴槽水栓さし湯時における太陽熱補正給湯負荷 (MJ/h)
    :return: 1日当たりの浴槽水栓さし湯時における太陽熱補正給湯負荷 (MJ/d)
    """
    return np.sum(L_dashdash_ba1_d_t.reshape((365, 24)), axis=1)


# 1日当たりの浴槽追焚時における太陽熱補正給湯負荷 (MJ/d)
def get_L_dashdash_ba2_d(L_dashdash_ba2_d_t):
    """
    :param L_dashdash_ba2_d_t: 1時間当たりの浴槽追焚時における太陽熱補正給湯負荷 (MJ/h)
    :return: 1日当たりの浴槽追焚時における太陽熱補正給湯負荷 (MJ/d)
    """
    return np.sum(L_dashdash_ba2_d_t.reshape((365, 24)), axis=1)